In [1]:
import numpy as np
import pandas as pd
import time
import ast
import json
import datetime
import pprint


import warnings 
warnings.filterwarnings('ignore')

import logging
from kiteconnect import KiteConnect

from kiteconnect import KiteConnect, KiteTicker
import time
import threading

logging.basicConfig(level=logging.ERROR)

from kiteconnect import KiteConnect
import os
import pickle

pd.options.display.max_rows = 500
pd.options.display.max_columns = 1000

In [1]:
kite = KiteConnect(api_key=api_key)
data = kite.generate_session(request_token, api_secret=api_secret)
access_token = data["access_token"]
kite.set_access_token(access_token)

In [1]:
instruments = kite.instruments()



In [2]:
inst_list = [
             "ICICIBANK","HDFCBANK","YESBANK","AXISBANK",\
             "INFY","TCS",\
             "TATAMOTORS","TVSMOTOR"\
            ]


In [1]:
dict_inst ={}
inst_dict = {} 
inst_ticks = {}
for inst in instruments:
    if inst.get("instrument_type") == 'EQ' and  inst.get("exchange") == "NSE":
        symbol = inst.get("tradingsymbol")
        dict_inst[symbol] = inst.get("instrument_token")
        inst_dict[inst.get("instrument_token")] = symbol
        inst_ticks[symbol] = inst.get("tick_size")
        if symbol in inst_list:
            print(inst)

In [28]:
print('last 5 days')
returns_dict ={}
delta_dict = {}
vol_dict = {}
for symbol in  inst_list:
    instrument_token = dict_inst.get(symbol)
    interval = "5minute"

    to_date = datetime.date.today()
    from_date =  to_date - datetime.timedelta(days=5)

    data = kite.historical_data(
        instrument_token=instrument_token,
        from_date=from_date,
        to_date=to_date,
        interval=interval,
        continuous=False,
        oi=False
    )

    close_prices = []
    print("--------------",symbol)
    
    df = pd.DataFrame(data)
    df['ticker'] =symbol
    df.to_csv(f'{symbol}.csv',index=False)
    


last 5 days
-------------- ICICIBANK
-------------- HDFCBANK
-------------- YESBANK
-------------- AXISBANK
-------------- INFY
-------------- TCS
-------------- TATAMOTORS
-------------- TVSMOTOR


## Plan of Action

- For each ticker, the opening range is constructed
- Based on opening bullish/bearish chandle, the breakout level is determined. Scope to use retracement levels
- Based on Fibonacci Golden ratio Extension , target is determined


In [ ]:
dfs= {}
for symbol in  inst_list:
    df = pd.read_csv(f'{symbol}.csv')

    df['dt'] = pd.to_datetime(df.date).dt.date

    bullish_df= df.groupby('dt').first().apply(lambda row:row['open']-row['close'] > 0,axis=1).to_frame().reset_index()

    bullish_df = bullish_df.rename({0:'bullish'},axis=1)

    OR_df = df.groupby('dt').first().apply(lambda row:row['high']-row['low'],axis=1).to_frame().reset_index()

    OR_df = OR_df.rename({0:'OR'},axis=1)
    
    
    df = df.merge(bullish_df).merge(OR_df)

    ORB_Target_df = df.groupby('dt').first().apply(lambda row: row['low'] + 1.618 *  row['OR'] if row['bullish'] else row['high'] - 1.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_Target_df = ORB_Target_df.rename({0:'ORB_golden_ratio_target'},axis=1)

    ORB_breakout_df = df.groupby('dt').first().apply(lambda row: row['high'] if row['bullish'] else row['low'],axis=1).to_frame().reset_index()

    ORB_breakout_df= ORB_breakout_df.rename({0:'ORB_breakout_level'},axis=1)

    df = df.merge(ORB_Target_df).merge(ORB_breakout_df)
    dfs[symbol]=df

In [ ]:
dfs.get(inst_list[0]).head(0)

### Parsing University Data

In [3]:
import os

data_dir = "E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/"
dirs =[] 
for p,q,r in os.walk(data_dir):
    if not r:
    
        [dirs.append(os.path.join(p,d)) for d in q]
        

In [4]:
dirs

['E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2021',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data January 2022\\Cash IEOD Gold - 2201',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data February 2022\\Cash IEOD Gold - 2202',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data March 2022\\Cash IEOD Gold - 2203',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data April 2022\\Cash Data April 2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data May 2022\\Cash Data May 2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data June 2022\\Cash Data June 2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash D

In [5]:
dfs ={}

for data_dir in dirs:
    for file in os.listdir(data_dir):
        if not file.endswith('.csv'):
            continue
        if file[0]!=".":
            symbol = file.removesuffix(".csv") 
            if symbol in inst_list :
#                 print(file)
                df = pd.read_csv(data_dir+"/"+file)
                df.columns = [ x.strip()[1:-1] for x in df.columns.tolist() ]
                df.date = pd.to_datetime(df.date)
                prev_df = dfs.get(symbol)
                if prev_df is not None:
                    df = pd.concat([prev_df,df],axis =0 )
                dfs[symbol]=df

In [6]:
## Sort by date , time and construct 5 min ohlc intervals
for k,v in dfs.items():
    v = v.sort_values(['date','time'],ascending=True)
    v['ohlc_interval'] = pd.to_datetime(v.time).dt.floor('5min').dt.time
    v['o'] = v.groupby(['date','ohlc_interval'])['open'].transform('first')
    v['h'] = v.groupby(['date','ohlc_interval'])['high'].transform('max')
    v['l'] = v.groupby(['date','ohlc_interval'])['low'].transform('min')
    v['c'] = v.groupby(['date','ohlc_interval'])['close'].transform('last')
    v['v'] = v.groupby(['date','ohlc_interval'])['volume'].transform('sum')
    v = v.drop_duplicates(['date','ohlc_interval'])
    dfs[k] = v

In [7]:
dfs_orb= {}
for symbol in  inst_list:
    df = dfs.get(symbol)

    df['dt'] = pd.to_datetime(df.date).dt.date

    bullish_df= df.groupby('dt').first().apply(lambda row:row['open']-row['close'] > 0,axis=1).to_frame().reset_index()

    bullish_df = bullish_df.rename({0:'bullish'},axis=1)

    OR_df = df.groupby('dt').first().apply(lambda row:row['high']-row['low'],axis=1).to_frame().reset_index()

    OR_df = OR_df.rename({0:'OR'},axis=1)
    
    
    df = df.merge(bullish_df).merge(OR_df)

    ORB_Target1_df = df.groupby('dt').first().apply(lambda row: row['low'] + 1.618 *  row['OR'] if row['bullish'] else row['high'] - 1.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_Target1_df = ORB_Target1_df.rename({0:'ORB_golden_ratio_target1'},axis=1)
    
    ORB_Target2_df = df.groupby('dt').first().apply(lambda row: row['low'] + 2.618 *  row['OR'] if row['bullish'] else row['high'] - 2.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_Target2_df = ORB_Target2_df.rename({0:'ORB_golden_ratio_target2'},axis=1)

    ORB_breakout_df = df.groupby('dt').first().apply(lambda row: row['high'] if row['bullish'] else row['low'],axis=1).to_frame().reset_index()

    ORB_breakout_df= ORB_breakout_df.rename({0:'ORB_breakout_level'},axis=1)
    
    ORB_entry_level_df = df.groupby('dt').first().apply(lambda row: row['high'] - 0.618 *  row['OR'] if row['bullish'] else row['low'] + 0.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_entry_level_df= ORB_entry_level_df.rename({0:'ORB_entry_level'},axis=1)
    
    ORB_stoploss_level_df = df.groupby('dt').first().apply(lambda row: row['high'] - 1 *  row['OR'] if row['bullish'] else row['low'] + 1 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_stoploss_level_df= ORB_stoploss_level_df.rename({0:'ORB_stoploss_level'},axis=1)

    df = df.merge(ORB_breakout_df).merge(ORB_entry_level_df).merge(ORB_stoploss_level_df).merge(ORB_Target1_df).merge(ORB_Target2_df)    
    
    df['did_break_out'] =df.apply(lambda x:x['h'] > x['ORB_breakout_level'] if x['bullish']  else x['l'] < x['ORB_breakout_level'],axis=1)
    df['did_retrace'] =df.apply(lambda x:x['l'] < x['ORB_entry_level'] if x['bullish']  else x['h'] > x['ORB_entry_level'],axis=1)
    
    dfs_orb[symbol]=df

In [8]:
dfs_orb.get(inst_list[0]).head(2)

,ticker,date,time,open,high,low,close,volume,o/i,ohlc_interval,o,h,l,c,v,dt,bullish,OR,ORB_breakout_level,ORB_entry_level,ORB_stoploss_level,ORB_golden_ratio_target1,ORB_golden_ratio_target2,did_break_out,did_retrace
0,ICICIBANK,2021-01-01,09:15:00,535.55,537.0,535.05,535.05,119643,0,09:15:00,535.55,537.0,534.75,535.75,313448,2021-01-01,True,1.95,537.0,535.7949,535.05,538.2051,540.1551,False,True
1,ICICIBANK,2021-01-01,09:20:00,535.80,535.8,535.05,535.50,86669,0,09:20:00,535.80,535.8,534.00,534.00,215648,2021-01-01,True,1.95,537.0,535.7949,535.05,538.2051,540.1551,False,True


In [9]:
icici_bank = dfs_orb.get(inst_list[0])

In [10]:
icici_bank.query('did_retrace & did_break_out').head(10)

,ticker,date,time,open,high,low,close,volume,o/i,ohlc_interval,o,h,l,c,v,dt,bullish,OR,ORB_breakout_level,ORB_entry_level,ORB_stoploss_level,ORB_golden_ratio_target1,ORB_golden_ratio_target2,did_break_out,did_retrace
79,ICICIBANK,2021-01-04,09:25:00,531.05,531.35,530.75,530.90,29227,0,09:25:00,531.05,533.30,530.30,533.25,383492,2021-01-04,True,1.50,532.55,531.6230,531.05,533.4770,534.9770,True,True
151,ICICIBANK,2021-01-04,15:25:00,532.45,533.00,532.45,532.90,53367,0,15:25:00,532.45,533.00,531.25,531.70,215313,2021-01-04,True,1.50,532.55,531.6230,531.05,533.4770,534.9770,True,True
155,ICICIBANK,2021-01-05,09:25:00,524.30,525.35,524.10,525.20,155370,0,09:25:00,524.30,527.40,524.10,526.75,717521,2021-01-05,True,3.65,526.65,524.3943,523.00,528.9057,532.5557,True,True
644,ICICIBANK,2021-01-13,11:30:00,552.60,553.00,552.45,552.95,89541,0,11:30:00,552.60,553.00,550.45,551.00,654468,2021-01-13,False,1.70,550.80,551.8506,552.50,549.7494,548.0494,True,True
645,ICICIBANK,2021-01-13,11:35:00,550.95,552.40,550.95,551.25,140156,0,11:35:00,550.95,552.40,550.50,550.70,483359,2021-01-13,False,1.70,550.80,551.8506,552.50,549.7494,548.0494,True,True
647,ICICIBANK,2021-01-13,11:45:00,551.10,551.35,550.55,550.55,98221,0,11:45:00,551.10,553.35,550.50,553.05,318480,2021-01-13,False,1.70,550.80,551.8506,552.50,549.7494,548.0494,True,True
650,ICICIBANK,2021-01-13,12:00:00,552.55,552.55,550.60,551.05,96943,0,12:00:00,552.55,552.55,550.60,551.85,201558,2021-01-13,False,1.70,550.80,551.8506,552.50,549.7494,548.0494,True,True
676,ICICIBANK,2021-01-13,14:10:00,549.80,550.70,549.75,550.55,36174,0,14:10:00,549.80,552.70,549.75,552.20,272909,2021-01-13,False,1.70,550.80,551.8506,552.50,549.7494,548.0494,True,True
771,ICICIBANK,2021-01-15,09:15:00,550.00,550.00,548.05,549.00,217507,0,09:15:00,550.00,550.95,548.05,549.40,608039,2021-01-15,True,1.95,550.00,548.7949,548.05,551.2051,553.1551,True,True
772,ICICIBANK,2021-01-15,09:20:00,549.50,549.85,548.70,549.85,63972,0,09:20:00,549.50,550.45,548.70,549.90,314249,2021-01-15,True,1.95,550.00,548.7949,548.05,551.2051,553.1551,True,True
